# Auxiliar 1 : Claisficación de texto


----------------------------------

## Breve resumen de las clases anteriores


### ¿En qué consiste la clasificación de texto?

Según [wikipedia](https://en.wikipedia.org/wiki/Document_classification): 

    "The task is to assign a document to one or more classes or categories"


Algunos ejemplos:

- Assigning subject categories, topics, or genres
- Spam detection 
- Authorship identification
- Age/gender identification
- Language Identification
- Sentiment analysis
- ...

### Definición formal

Input:
- A document    $d$
- A fixed set of classes $C=\{c_{1},    c_{2},...,    c_{J}\}$

Output:    

- A predicted class $c \in C$

### Tipos de técnicas de clasificación: 

- Hand-coded Rules. (No se verán en este aux).
- Supervised Machine Learning: 
    - Naïve Bayes
    - Logistic regression
    - Support vector machines
    - k-Nearest Neighbors


## Objetivo del Auxiliar

Introducirlos en los primeros tópicos y herramientas comunes de NLP.

Para esto, implementaremos variados modelos de clasificación de texto destinadas a **predecir la categoría de noticias de la radio biobio**.

Las tecnicas que usaremos serán las vistas en clases: 

- Bayes
- Logistic regression 

Las herramientas que usaremos son (y qué serán **necesarias** para ejecutar este notebook): 

- Pandas
- Scikit-Learn
- Spacy
- NLTK



## Créditos

Todas las noticias extraidas perteneces a [Biobio Chile](https://www.biobiochile.cl/), los cuales gentilmente licencian todo su material a través de la [licencia Creative Commons (CC-BY-NC)](https://creativecommons.org/licenses/by-nc/2.0/cl/)

## Referencias: 

Gitgub del curso: 
- https://github.com/dccuchile/CC6205

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Códigos varios:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)

## Imports

In [1]:
import pandas as pd    
import spacy
import nltk

from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report

from nltk.stem import SnowballStemmer
from spacy.lang.es.stop_words import STOP_WORDS

nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Datos

### Cargar los datasets 

In [2]:
nacional = pd.read_json("./datasets/biobio_nacional.json", encoding ='utf-8')
internacional = pd.read_json("./datasets/biobio_internacional.json", encoding ='utf-8')
economia = pd.read_json("./datasets/biobio_economia.json", encoding ='utf-8')
sociedad = pd.read_json("./datasets/biobio_sociedad.json", encoding ='utf-8')
opinion = pd.read_json("./datasets/biobio_opinion.json", encoding ='utf-8')

In [3]:
sociedad.describe()

,publication_date,publication_hour,author,author_link,title,link,category,subcategory,content,tags,embedded_links
count,200,200,200,200,200,200,200,200,200,200,200
unique,42,167,13,13,200,200,1,1,196,191,3
top,26/06/2019,15:12,César Vega Martínez,/lista/autores/cevega,Hombre irrumpe de noche en estación de policía...,https://www.biobiochile.cl/noticias/sociedad/m...,Sociedad,Sociedad,,[],[]
freq,10,3,78,78,1,1,200,200,5,9,198


#### Ejemplo de noticia de categoría sociedad: 

In [4]:
sample = sociedad.iloc[19:20]

In [5]:
sample

,publication_date,publication_hour,author,author_link,title,link,category,subcategory,content,tags,embedded_links
19,02/08/2019,15:08,Emilio Contreras,/lista/autores/Econtreras,Chile deja de utilizar 16.170 toneladas de bol...,https://www.biobiochile.cl/noticias/sociedad/d...,Sociedad,Sociedad,Chile dejó de utilizar 16.170 toneladas de b...,"[#16.170 toneladas, #balance, #bolsas plástica...",[]


In [6]:
sample_content = sample.values[0][8]
sample_category = sample.values[0][6]

In [7]:
print("\033[1mContenido:\033[0m\n\n", sample_content.strip(), "\n\n\033[1mClase:\033[0m\n\n", sample_category)

Contenido:

 Chile dejó de utilizar 16.170 toneladas de bolsas plásticas desde que hace un año implementó una ley que prohíbe su entrega en supermercados y retails, informó este viernes el ministerio del Medio Ambiente.  Tras un periodo de prueba de seis meses, el gobierno chileno puso en vigencia en agosto del año pasado la ley que evitó el consumo de unas 2.200 millones de bolsas plásticas, una reducción significativa tomando en cuenta que, hasta la promulgación de la norma, Chile producía 3.200 millones de bolsas anuales .  “Si consideramos el peso de estas bolsas que se dejaron de entregar, unas 16.170 toneladas, equivalen a 13.940 autos”, indicó el ministerio en un comunicado.  Desde la puesta en marcha de la norma, los chilenos asumieron el hábito de utilizar bolsas reutilizables, de tela o material reciclable, lo cual ha colaborado en reducir la contaminación que producen los sacos plásticos principalmente en los océanos, en los que yacen unos 13 millones de toneladas de plástic

### Tokenizar

¿Qué era tokenizar?

    In computer science, lexical analysis, lexing or tokenization is the process of converting a sequence of characters (such as in a computer program or web page) into a sequence of tokens (strings with an assigned and thus identified meaning).
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)

In [8]:
tokenized_content = [word.text for word in  nlp(sample_content)]

In [9]:
pd.DataFrame(tokenized_content)

,0
0,
1,Chile
2,dejó
3,de
4,utilizar
5,16.170
6,toneladas
7,de
8,bolsas
9,plásticas


### Stopwords 

¿Qué eran las stopwords?

    In computing, stop words are words which are filtered out before or after processing of natural language data (text).[1] Stop words are generally the most common words in a language, there is no single universal list of stop words used by all natural language processing tools, and indeed not all tools even use such a list. Some tools avoid removing stop words to support phrase search. 
    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

In [10]:
STOP_WORDS

{'actualmente',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'adrede',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún',
 'alli',
 'allí',
 'alrededor',
 'ambos',
 'ampleamos',
 'antano',
 'antaño',
 'ante',
 'anterior',
 'antes',
 'apenas',
 'aproximadamente',
 'aquel',
 'aquella',
 'aquellas',
 'aquello',
 'aquellos',
 'aqui',
 'aquél',
 'aquélla',
 'aquéllas',
 'aquéllos',
 'aquí',
 'arriba',
 'arribaabajo',
 'aseguró',
 'asi',
 'así',
 'atras',
 'aun',
 'aunque',
 'ayer',
 'añadió',
 'aún',
 'bajo',
 'bastante',
 'bien',
 'breve',
 'buen',
 'buena',
 'buenas',
 'bueno',
 'buenos',
 'cada',
 'casi',
 'cerca',
 'cierta',
 'ciertas',
 'cierto',
 'ciertos',
 'cinco',
 'claro',
 'comentó',
 'como',
 'con',
 'conmigo',
 'conocer',
 'conseguimos',
 'conseguir',
 'considera',
 'consideró',
 'consigo',
 'consigue',
 'consiguen',
 'consigues',
 'contigo',
 'contra',
 'cosas',
 'creo',
 'cual',
 'cuales',
 'cualquier',
 

In [11]:
tokenized_content_no_stop_words = [token for token in tokenized_content if token not in STOP_WORDS ]

In [12]:
pd.DataFrame(tokenized_content_no_stop_words)

,0
0,
1,Chile
2,utilizar
3,16.170
4,toneladas
5,bolsas
6,plásticas
7,año
8,implementó
9,ley


### Stemming

¿Qué era el stemming? 

    Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form—generally a written word form.
    
Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
#### Ejemplos: 


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

In [13]:
stemmer = SnowballStemmer('spanish')
stemmed_content = [stemmer.stem(word) for word in tokenized_content]

In [14]:
pd.DataFrame(zip(tokenized_content, stemmed_content), columns=['original', 'stem']).sample(15)

,original,stem
98,el,el
253,pequeños,pequeñ
54,del,del
302,2025,2025
135,",",","
146,de,de
29,viernes,viern
304,en,en
350,",",","
285,de,de


### Lematización

¿Qué era lematización? 

    
    Lemmatisation (or lemmatization) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.[
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)
    
#### Ejemplos

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


In [15]:
lemmatized_content = [word.lemma_ for word in nlp(sample_content)]

In [16]:
# Visualizar la lematización
pd.DataFrame(zip(tokenized_content, lemmatized_content), columns=['original', 'lemma']).sample(15)

,original,lemma
71,una,uno
216,en,en
146,de,de
155,colaborado,colaborar
197,,
88,3.200,3.200
194,gratis,gratis
123,un,uno
109,unas,uno
215,hacer,hacer


### Bag of Words

¿Qué es?


    
    The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words 
    
Referencia: [BoW en wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)

### Ejemplo

- Doc1 : 'I love dogs'
- Doc2: 'I hate dogs and knitting.
- Doc3: 'Knitting is my hobby and my passion.

![BOW](https://i1.wp.com/datameetsmedia.com/wp-content/uploads/2017/05/bagofwords.004.jpeg?resize=1024%2C260)

In [17]:
# Tokenizers for CountVectorizer

def tokenizer(doc):
    return [x.orth_ for x in nlp(doc) ]
    
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]

def tokenizer_with_lemmatization (doc):
    return [x.lemma_ for x in nlp(doc)]
  
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

In [18]:
vectorizer = CountVectorizer(analyzer='word', tokenizer = tokenizer, ngram_range=(1,2))  
bow = vectorizer.fit_transform(opinion.sample(4).content)
bow[0]

<1x2023 sparse matrix of type '<class 'numpy.int64'>'
	with 656 stored elements in Compressed Sparse Row format>

## Procesar los datasets

Seleccionar solo las columnas relevantes y divider en conjuntos de entrenamiento y de prueba.

In [19]:
def process_datasets(datasets):
    dataset = pd.concat(datasets)
    X_train, X_test, y_train, y_test = train_test_split(dataset.content, dataset.category, test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [20]:
datasets = [nacional, internacional, economia, sociedad, opinion]
X_train, X_test, y_train, y_test = process_datasets(datasets)

## Clasificación con Naive Bayes

- Simple (“naïve”) classification method based on Bayes rule
- Relies on very simple representation of document
- Bag of words

Para un document d y la clase C, la probabilidad de está dada por:

$$P(c|d) = \frac{P(d|c)P(c)}{P(d)}$$

Consideremos MAP como Maximum a posteriori o la clase mas probable. 

$$ C_{MAP} = argmax_{c \in C} P(c|d)$$

Aplicando el teorema de Bayes:

$$ C_{MAP} = argmax_{c \in C} \frac{P(d|c)P(c)}{P(d)} $$

Descartamos el denominador:

$$ C_{MAP} = argmax_{c \in C} P(d|c)P(c) $$

Si el documento d, ahora lo consideramos como un arreglo de palabras (pensando en bag of words): 

$$ C_{MAP} = argmax_{c\in C} P(x_1, x_2, ..., x_n | c)P(c) $$ 

El clasificador aprenderá $O(|X|^n * |C|)$ parámetros, los cuales deberán ser entrenados a partir de una gran cantidad de ejemplos de entrenamiento.

### Establecer el Pipeline

In [21]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

vectorizer = CountVectorizer(analyzer='word', tokenizer = TOKENIZER, ngram_range=(1,3))  
clf = MultinomialNB()   

text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

### Entrenar

In [22]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_with_lemmatization at 0x00000232CE393840>,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

### Evaluación

In [23]:
predicted = text_clf.predict(X_test)

conf = confusion_matrix(y_test, predicted)
kappa = cohen_kappa_score(y_test, predicted) 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nkappa:'+str(kappa))



Confusion Matrix for Logistic Regression + ngram features:
[[56  0  2 11  1]
 [ 0 64  1  2 10]
 [ 1  0 37 19  1]
 [ 0  0  0 66  0]
 [ 0  5  0  3 51]]

Classification Report
               precision    recall  f1-score   support

     Economia       0.98      0.80      0.88        70
Internacional       0.93      0.83      0.88        77
     Nacional       0.93      0.64      0.76        58
      Opinion       0.65      1.00      0.79        66
     Sociedad       0.81      0.86      0.84        59

     accuracy                           0.83       330
    macro avg       0.86      0.83      0.83       330
 weighted avg       0.86      0.83      0.83       330


kappa:0.7873270881763988


### Ejemplos

In [24]:
text_clf.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

array(['Sociedad'], dtype='<U13')

In [25]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['Internacional'], dtype='<U13')

In [26]:
text_clf.predict(["El banco mundial presentó para chile un decrecimiento económico de 92% y una inflación de 8239832983289%."])

array(['Economia'], dtype='<U13')

## Regresión Logísitica

Explicación cuática aquí

### Pipeline

In [27]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

### Entrenar

In [28]:
log_pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_with_lemmatization at 0x00000232CE393840>,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
    

### Evaluar

In [29]:
predicted = log_pipe.predict(X_test)

conf = confusion_matrix(y_test, predicted)
kappa = cohen_kappa_score(y_test, predicted) 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nkappa:'+str(kappa))


Confusion Matrix for Logistic Regression + ngram features:
[[58  1  6  1  4]
 [ 5 61  5  1  5]
 [ 2  0 53  1  2]
 [ 1  1  2 61  1]
 [ 2  6  2  1 48]]

Classification Report
               precision    recall  f1-score   support

     Economia       0.85      0.83      0.84        70
Internacional       0.88      0.79      0.84        77
     Nacional       0.78      0.91      0.84        58
      Opinion       0.94      0.92      0.93        66
     Sociedad       0.80      0.81      0.81        59

     accuracy                           0.85       330
    macro avg       0.85      0.85      0.85       330
 weighted avg       0.85      0.85      0.85       330


kappa:0.8142510884174009


### Ejemplos

In [30]:
log_pipe.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

array(['Sociedad'], dtype=object)

In [31]:
log_pipe.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['Sociedad'], dtype=object)

## Clasificación de subcategorías

In [32]:
pd.concat(datasets).subcategory.unique()

array(['Nacional', 'Región Metropolitana', 'Región del Bío Bío',
       'Región de los Ríos', 'Región de los Lagos',
       'Región de Valparaíso', 'Región de Antofagasta',
       'Región del Maule', 'Región de la Araucanía', 'Región de Ñuble',
       "Región de O'Higgins", 'Región de Aysén', 'Región de Coquimbo',
       'Región de Tarapacá', 'Región de Magallanes', 'Región de Atacama',
       'Internacional', 'Economía', 'Sociedad', 'Opinión'], dtype=object)

In [33]:
def process_datasets_with_subcategories(datasets):
    dataset = pd.concat(datasets)
    X_train, X_test, y_train, y_test = train_test_split(dataset.content, dataset.subcategory, test_size=0.33, random_state=42)    
    return X_train, X_test, y_train, y_test
    

In [34]:
X_train, X_test, y_train, y_test = process_datasets_with_subcategories(datasets)

In [35]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod_2 = LogisticRegression(solver='lbfgs', multi_class='ovr', v)   
log_pipe_2 = Pipeline([('vect', vectorizer), ('clf', log_mod_2)])

SyntaxError: positional argument follows keyword argument (<ipython-input-35-1dd2234e9ec4>, line 4)

In [ ]:
log_pipe_2.fit(X_train, y_train)

In [ ]:
predicted = log_pipe_2.predict(X_test)

conf = confusion_matrix(y_test, predicted)
kappa = cohen_kappa_score(y_test, predicted) 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nkappa:'+str(kappa))